## 1. Consultas sobre redis (local)

In [43]:
# Conexion a redis 
import redis as r

REDIS_PORT = 6379
r = r.Redis(host='localhost',
                port=REDIS_PORT,
                decode_responses=True)

In [44]:
# Para comprobar cuanta memoria usada, habria que utilizarla tanto por maestro como replicas  
mem = r.info("memory")
used_mb = mem["used_memory"] / 1024 / 1024
total_mb = mem.get("total_system_memory", 0) / 1024 / 1024

print(f"Usada: {used_mb:.2f} MB")
print(f"Total sistema: {total_mb:.2f} MB")

Usada: 1.08 MB
Total sistema: 7567.28 MB


### 1.1 Colas de pacientes

In [54]:
# Scan de las colas de pacientes
zsets = r.scan(match="ED-*:DEP-*",
       _type="ZSET")[1]
for key in zsets:
       print(key)

ED-3:DEP-ONCOLOGIA
ED-4:DEP-NEUROLOGIA
ED-3:DEP-CARDIOLOGIA
ED-2:DEP-PEDIATRIA
ED-2:DEP-ONCOLOGIA


In [61]:
# Comprobar el contenido de las colas añadidas:
for key in zsets:
    print(f"Contenido de: {key}")
    scan_results = r.zscan(key,match="*")
    print(f"  Database --> {scan_results[0]}",end="\n    ")
    for patient in scan_results[1]:
        print(patient, end=" ")
    print("\n")

Contenido de: ED-3:DEP-ONCOLOGIA
  Database --> 0
    ('Daniel', 1.0) ('Mariana', 27.0) ('Mateo', 32.0) ('Alejandro', 41.0) ('Camila', 82.0) ('Diego', 92.0) ('Lucas', 93.0) ('Sofía', 99.0) 

Contenido de: ED-4:DEP-NEUROLOGIA
  Database --> 0
    ('Isabella', 35.0) ('Javier', 38.0) ('Daniel', 43.0) ('Mateo', 45.0) ('Lucas', 58.0) ('Valentina', 83.0) ('Mariana', 89.0) ('Alejandro', 100.0) 

Contenido de: ED-3:DEP-CARDIOLOGIA
  Database --> 0
    ('Lucía', 0.0) ('Diego', 27.0) ('Mateo', 30.0) ('Isabella', 38.0) ('Javier', 50.0) ('Martina', 59.0) ('Andrés', 61.0) ('Sofía', 86.0) 

Contenido de: ED-2:DEP-PEDIATRIA
  Database --> 0
    ('Valentina', 4.0) ('Daniel', 12.0) ('Camila', 25.0) ('Martina', 57.0) ('Mariana', 63.0) ('Alejandro', 67.0) ('Mateo', 67.0) ('Diego', 72.0) 

Contenido de: ED-2:DEP-ONCOLOGIA
  Database --> 0
    ('Lucas', 14.0) ('Javier', 27.0) ('Mariana', 42.0) ('Andrés', 54.0) ('Isabella', 64.0) ('Sofía', 72.0) ('Lucía', 91.0) ('Martina', 95.0) 



In [80]:
# Pacientes prioritarios en cada cola 
for key in zsets: 
    print(key)
    [(patient, score)] = r.zrevrange(key, start=0, end=0, withscores=True)
    print(f"-> Paciente: {patient} | Prioridad: {score}")

ED-3:DEP-ONCOLOGIA
-> Paciente: Sofía | Prioridad: 99.0
ED-4:DEP-NEUROLOGIA
-> Paciente: Alejandro | Prioridad: 100.0
ED-3:DEP-CARDIOLOGIA
-> Paciente: Sofía | Prioridad: 86.0
ED-2:DEP-PEDIATRIA
-> Paciente: Diego | Prioridad: 72.0
ED-2:DEP-ONCOLOGIA
-> Paciente: Martina | Prioridad: 95.0


## 2. Consultas sobre redis cloud 